# QNN baseline
Simple descriptor-based quantum regression baseline on a simulator.


In [ ]:
!pip -q install rdkit-pypi pennylane torch pandas numpy scikit-learn matplotlib


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

train = pd.read_csv("../data/train.csv")
val = pd.read_csv("../data/val.csv")
test = pd.read_csv("../data/test.csv")


In [ ]:
DESC_FUNCS = [
    Descriptors.MolWt,
    Descriptors.MolLogP,
    Descriptors.TPSA,
    Descriptors.NumHDonors,
    Descriptors.NumHAcceptors,
    Descriptors.NumRotatableBonds,
    Descriptors.RingCount,
    Descriptors.FractionCSP3,
]  # 8 features = 8 qubits

def featurize(smiles):
    m = Chem.MolFromSmiles(smiles)
    if m is None:
        return None
    v = [float(f(m)) for f in DESC_FUNCS]
    if any([np.isnan(x) or np.isinf(x) for x in v]):
        return None
    return v

def frame_to_Xy(df):
    feats, ys = [], []
    for s, y in zip(df.SMILES, df.Solubility):
        v = featurize(s)
        if v is not None:
            feats.append(v)
            ys.append(float(y))
    return np.array(feats, np.float32), np.array(ys, np.float32)

Xtr, ytr = frame_to_Xy(train)
Xva, yva = frame_to_Xy(val)
Xte, yte = frame_to_Xy(test)

sc = StandardScaler().fit(Xtr)
Xtr = sc.transform(Xtr)
Xva = sc.transform(Xva)
Xte = sc.transform(Xte)


In [ ]:
import pennylane as qml

NQ = Xtr.shape[1]
LAYERS = 2
dev = qml.device("default.qubit", wires=NQ, shots=None)

def feature_encoding(x):
    for i in range(NQ):
        qml.RX(x[i], wires=i)
        qml.RY(x[i], wires=i)

def variational_block(params):
    for i in range(NQ):
        qml.RY(params[i], wires=i)
    for i in range(NQ - 1):
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dev, interface="torch")
def qnode(x, thetas):
    feature_encoding(x)
    for l in range(LAYERS):
        variational_block(thetas[l])
    return qml.expval(qml.PauliZ(0))

class QNNReg(nn.Module):
    def __init__(self):
        super().__init__()
        self.thetas = nn.Parameter(torch.randn(LAYERS, NQ) * 0.1)
        self.alpha = nn.Parameter(torch.tensor(1.0))
        self.beta = nn.Parameter(torch.tensor(0.0))

    def forward(self, x):
        outs = [qnode(x[i], self.thetas).float() for i in range(x.shape[0])]
        out = torch.stack(outs)
        return self.alpha * out + self.beta


In [ ]:
def train_epoch(model, X, y, opt, crit, bs=64):
    model.train()
    order = np.random.permutation(len(X))
    losses = []
    for i in range(0, len(X), bs):
        idx = order[i : i + bs]
        xb = torch.tensor(X[idx], dtype=torch.float32)
        yb = torch.tensor(y[idx], dtype=torch.float32)
        opt.zero_grad()
        pred = model(xb)
        loss = crit(pred, yb)
        loss.backward()
        opt.step()
        losses.append(loss.item())
    return float(np.mean(losses))

@torch.no_grad()
def evaluate_q(model, X, y):
    xb = torch.tensor(X, dtype=torch.float32)
    pred = model(xb).float().detach().numpy()
    rmse = float(np.sqrt(((y - pred) ** 2).mean()))
    mae = float(np.mean(np.abs(y - pred)))
    r2 = float(r2_score(y, pred))
    return rmse, mae, r2, pred

model = QNNReg()
opt = torch.optim.Adam(model.parameters(), lr=1e-2)
crit = nn.MSELoss()

best = (1e9, None)
for ep in range(1, 81):
    tr_loss = train_epoch(model, Xtr, ytr, opt, crit)
    rmse, mae, r2, _ = evaluate_q(model, Xva, yva)
    if rmse < best[0]:
        best = (rmse, {k: v.detach().clone() for k, v in model.state_dict().items()})
    if ep % 10 == 0:
        print(f"ep{ep:03d} loss {tr_loss:.4f} | val RMSE {rmse:.3f} R2 {r2:.3f}")

if best[1] is not None:
    model.load_state_dict(best[1])

rmse, mae, r2, pred_te = evaluate_q(model, Xte, yte)
print("QNN test:", rmse, mae, r2)
pd.DataFrame({"y_true": yte, "y_pred": pred_te}).to_csv("../results/qnn_test_predictions.csv", index=False)


In [ ]:
# Visuals
plt.figure()
plt.scatter(yte, pred_te, s=12, alpha=0.5)
lims = [min(yte.min(), pred_te.min()), max(yte.max(), pred_te.max())]
plt.plot(lims, lims)
plt.xlabel("True")
plt.ylabel("Pred")
plt.title("QNN Parity")
plt.show()

plt.figure()
plt.hist(yte - pred_te, bins=40)
plt.title("QNN Residuals")
plt.show()
